**Лабораторная работа №9. Анализ шума**

In [ ]:
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
from math import factorial

import os
import itertools

from scipy import signal
from tqdm import tqdm

dpi = 1000

In [ ]:
def plot_spectrogram(fs, aud, name):
    powerSpectrum, frequenciesFound, time, imageAxis = plt.specgram(aud, Fs=fs)
    plt.ylabel('Частота [Гц]')
    plt.xlabel('Время [с]')
    plt.savefig('spectrogram_' + name + '.png')
    plt.clf()
    return powerSpectrum, frequenciesFound, time

# Функция для применения фильтра Савицкого-Голея
def savitzky_golay(fs, aud, cutoff_freuency, passes=1):
    z = signal.savgol_filter(aud, 100, 3)
    b, a = signal.butter(3, cutoff_freuency / fs)
    zi = signal.lfilter_zi(b, a)
    z, _ = signal.lfilter(b, a, z, zi = zi * z[0])
    return z

# Aильтр Баттерворта низких частот к входному сигналу
def butter_filter(samples, order):
    b, a = signal.butter(order, 0.1, btype='lowpass')
    filtered_signal = signal.filtfilt(b, a, samples)

    return filtered_signal

def find_peaks(spec, freq, t, delta_t=0.1, delta_freq=50):
    peaks = set()
    delta_t = 0.1
    delta_freq = 50

    t_increment = int(delta_t * len(t) / (t[-1] - t[0]))
    freq_increment = int(delta_freq * len(freq) / (freq[-1] - freq[0]))

    for i, freq_val in enumerate(freq):
        for j, time_val in enumerate(t):
            t_min = max(0, j - t_increment)
            t_max = min(len(t), j + t_increment + 1)
            freq_min = max(0, i - freq_increment)
            freq_max = min(len(freq), i + freq_increment + 1)

            is_peak = True
            for a in range(freq_min, freq_max):
                for b in range(t_min, t_max):
                    if (a != i or b != j) and spec[i, j] < spec[a, b]:
                        is_peak = False
                        break
                if not is_peak:
                    break

            if is_peak:
                peaks.add(time_val)

    return peaks

def to_pcm(y):
    return np.int16(y / np.max(np.abs(y)) * 32000)

def main():
    fs, aud = wav.read('gita.wav')
    aud = aud[:, 0]
    plot_spectrogram(fs, aud, 'original')
    wav.write('original.wav', fs, to_pcm(aud))

    denoised_savgol = savitzky_golay(fs, aud, 1000)
    plot_spectrogram(fs, denoised_savgol, 'denoised_savgol')
    wav.write('denoised_savgol.wav', fs, to_pcm(denoised_savgol))

    denoised_butter = butter_filter(denoised_savgol, 3)
    spec, freq, time = plot_spectrogram(fs, denoised_butter, 'denoised_butter')
    wav.write('denoised_butter.wav', fs, to_pcm(denoised_butter))

    peaks = find_peaks(spec, freq, time)
    with open('results.txt', 'w') as f:
        f.write("Количество пиков:" + str(len(peaks)) + '\n')
        for peak in peaks:
            f.write(str(peak) + '\n')

if __name__ == "__main__":
    main()

<Figure size 640x480 with 0 Axes>